In [1]:
!pip install -q tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 75kB/s 
     |████████████████████████████████| 501kB 53.8MB/s 
     |████████████████████████████████| 4.3MB 59.5MB/s 


In [0]:
import os
import time
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# Decompress the compressed reorganized "Jester" dataset in Google Colab VM

start = time.time()
!cat /content/gdrive/My\ Drive/DashGC/Reorganized/Dataset.tar.gz.?? | tar -xz
stop = time.time()
print('Decompression took', round(((stop - start) / 60), 2), 'mins')

Decompression took 9.27 mins


In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
import pathlib
train_data_root_orig = "./Dataset/Train"
train_data_root = pathlib.Path(train_data_root_orig)
print(train_data_root)

val_data_root_orig = "./Dataset/Validation"
val_data_root = pathlib.Path(val_data_root_orig)
print(val_data_root)

Dataset/Train
Dataset/Validation


In [6]:
for item in val_data_root.iterdir():
    print(item)

Dataset/Validation/Pushing Hand Away
Dataset/Validation/Sliding Two Fingers Down
Dataset/Validation/Sliding Two Fingers Right
Dataset/Validation/Swiping Left
Dataset/Validation/Doing other things
Dataset/Validation/Shaking Hand
Dataset/Validation/Swiping Down
Dataset/Validation/Thumb Up
Dataset/Validation/Turning Hand Counterclockwise
Dataset/Validation/Zooming Out With Two Fingers
Dataset/Validation/Stop Sign
Dataset/Validation/Swiping Up
Dataset/Validation/Sliding Two Fingers Up
Dataset/Validation/No gesture
Dataset/Validation/Rolling Hand Forward
Dataset/Validation/Zooming In With Two Fingers
Dataset/Validation/Turning Hand Clockwise
Dataset/Validation/Zooming Out With Full Hand
Dataset/Validation/Drumming Fingers
Dataset/Validation/Pulling Hand In
Dataset/Validation/Thumb Down
Dataset/Validation/Pushing Two Fingers Away
Dataset/Validation/Rolling Hand Backward
Dataset/Validation/Sliding Two Fingers Left
Dataset/Validation/Pulling Two Fingers In
Dataset/Validation/Zooming In With Fu

In [10]:
val_image_paths = list(val_data_root.glob('*/*/*'))
val_image_paths = [str(path) for path in val_image_paths]
val_image_paths.sort(key = lambda x: (int(str(x).split('/')[-1][:-5]), int(str(x).split('/')[-2])))

train_image_paths = list(train_data_root.glob('*/*/*'))
train_image_paths = [str(path) for path in train_image_paths]
train_image_paths.sort(key = lambda x: (int(str(x).split('/')[-1][:-5]), int(str(x).split('/')[-2])))

val_image_count = len(val_image_paths)
print(val_image_count)

train_image_count = len(train_image_paths)
print(train_image_count)

532578
4219150


In [14]:
print(val_image_paths[:10])
print(train_image_paths[:10])

['Dataset/Validation/Thumb Down/16/00003.jpg', 'Dataset/Validation/Thumb Down/16/00002.jpg', 'Dataset/Validation/Thumb Down/16/00001.jpg', 'Dataset/Validation/Thumb Down/16/00008.jpg', 'Dataset/Validation/Thumb Down/16/00005.jpg', 'Dataset/Validation/Thumb Down/16/00007.jpg', 'Dataset/Validation/Thumb Down/16/00006.jpg', 'Dataset/Validation/Thumb Down/16/00004.jpg', 'Dataset/Validation/Thumb Down/16/00009.jpg', 'Dataset/Validation/Zooming In With Full Hand/22/00003.jpg']
['Dataset/Train/Doing other things/1/00003.jpg', 'Dataset/Train/Doing other things/1/00002.jpg', 'Dataset/Train/Doing other things/1/00001.jpg', 'Dataset/Train/Doing other things/1/00008.jpg', 'Dataset/Train/Doing other things/1/00005.jpg', 'Dataset/Train/Doing other things/1/00007.jpg', 'Dataset/Train/Doing other things/1/00006.jpg', 'Dataset/Train/Doing other things/1/00004.jpg', 'Dataset/Train/Doing other things/1/00009.jpg', 'Dataset/Train/Pushing Two Fingers Away/3/00003.jpg']


In [15]:
label_names = sorted(item.name for item in train_data_root.glob('*/') if item.is_dir())
label_names

['Doing other things',
 'Drumming Fingers',
 'No gesture',
 'Pulling Hand In',
 'Pulling Two Fingers In',
 'Pushing Hand Away',
 'Pushing Two Fingers Away',
 'Rolling Hand Backward',
 'Rolling Hand Forward',
 'Shaking Hand',
 'Sliding Two Fingers Down',
 'Sliding Two Fingers Left',
 'Sliding Two Fingers Right',
 'Sliding Two Fingers Up',
 'Stop Sign',
 'Swiping Down',
 'Swiping Left',
 'Swiping Right',
 'Swiping Up',
 'Thumb Down',
 'Thumb Up',
 'Turning Hand Clockwise',
 'Turning Hand Counterclockwise',
 'Zooming In With Full Hand',
 'Zooming In With Two Fingers',
 'Zooming Out With Full Hand',
 'Zooming Out With Two Fingers']

In [16]:
label_to_index = {name: index for index, name in enumerate(label_names)}
label_to_index

{'Doing other things': 0,
 'Drumming Fingers': 1,
 'No gesture': 2,
 'Pulling Hand In': 3,
 'Pulling Two Fingers In': 4,
 'Pushing Hand Away': 5,
 'Pushing Two Fingers Away': 6,
 'Rolling Hand Backward': 7,
 'Rolling Hand Forward': 8,
 'Shaking Hand': 9,
 'Sliding Two Fingers Down': 10,
 'Sliding Two Fingers Left': 11,
 'Sliding Two Fingers Right': 12,
 'Sliding Two Fingers Up': 13,
 'Stop Sign': 14,
 'Swiping Down': 15,
 'Swiping Left': 16,
 'Swiping Right': 17,
 'Swiping Up': 18,
 'Thumb Down': 19,
 'Thumb Up': 20,
 'Turning Hand Clockwise': 21,
 'Turning Hand Counterclockwise': 22,
 'Zooming In With Full Hand': 23,
 'Zooming In With Two Fingers': 24,
 'Zooming Out With Full Hand': 25,
 'Zooming Out With Two Fingers': 26}

In [17]:
train_image_labels = [label_to_index[pathlib.Path(path).parent.parent.name]
                    for path in train_image_paths]

val_image_labels = [label_to_index[pathlib.Path(path).parent.parent.name]
                    for path in val_image_paths]

print("First 10 labels indices: ", train_image_labels[:10])

First 10 labels indices:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 6]


In [0]:
start = time.time()
print(min([tf.image.decode_image(tf.io.read_file(img_path)).shape[0] for img_path in train_image_paths]), max([tf.image.decode_image(tf.io.read_file(img_path)).shape[0] for img_path in train_image_paths]))
print(min([tf.image.decode_image(tf.io.read_file(img_path)).shape[0] for img_path in val_image_paths]), max([tf.image.decode_image(tf.io.read_file(img_path)).shape[0] for img_path in val_image_paths]))
print(min([tf.image.decode_image(tf.io.read_file(img_path)).shape[1] for img_path in train_image_paths]), max([tf.image.decode_image(tf.io.read_file(img_path)).shape[1] for img_path in train_image_paths]))
print(min([tf.image.decode_image(tf.io.read_file(img_path)).shape[1] for img_path in val_image_paths]), max([tf.image.decode_image(tf.io.read_file(img_path)).shape[1] for img_path in val_image_paths]))
stop = time.time()
print('Decompression took', round(((stop - start) / 60), 2), 'mins')

In [19]:
print(set(map(lambda x: x[-4:], train_image_paths)), set(map(lambda x: x[-4:], val_image_paths)))

{'.jpg'} {'.jpg'}


In [0]:
# Decode the video frames as int64 values of color & recast it as float32
# Resize the video frames into a standard size (100, 100, channels = 3)

#@tf.function
def preprocess_image(image):
    # img_raw = tf.io.read_file(img_path)
    # img_tensor = tf.image.decode_image(img_raw)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [100, 100])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [0]:
# Load the images as tensors (numpy-like multidimensional arrays)

train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

val_path_ds = tf.data.Dataset.from_tensor_slices(val_image_paths)
val_image_ds = val_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

In [0]:
train_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_image_labels, tf.int64))
val_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(val_image_labels, tf.int64))

train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
val_image_label_ds = tf.data.Dataset.zip((val_image_ds, val_label_ds))

In [0]:
tf.Keras.Layers.Input(shape = (70, 70, 3), sparse = True)

In [0]:
tf.ragged.constant()

In [0]:
img_raw = tf.io.read_file(img_path)
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

img_final = tf.image.resize(img_tensor, [100, 100])
img_final = img_final/255.0
print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())